In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain_community.embeddings import HuggingFaceHubEmbeddings
from langchain import hub
from langchain.vectorstores import Chroma
import pickle

Document loader

In [3]:
pages = PyPDFLoader("src/Constitution_de_la_republique_tunisiennefr.pdf").load_and_split()
len(pages)

19

Splitter


In [41]:
doc_chunks = []
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    separators=["\nArticle", "\n\n", "\n", ".", "!", "?", ",", " ", ""],
    chunk_overlap=50,
)
chunks = text_splitter.split_documents(pages)
# for i, chunk in enumerate(chunks):
#     doc = Document(
#         page_content=chunk, metadata={"page": doc.metadata["page"], "chunk": i}
#     )
#     doc.metadata["source"] = f"{doc.metadata['page']}-{doc.metadata['chunk']}"
#     doc.metadata["filename"] = "Constitution_de_la_republique_tunisienne" 
#     doc_chunks.append(doc)
len(chunks)

142

Vectorstores

In [4]:
# model =  SentenceTransformer("dangvantuan/sentence-camembert-large")

# sentences = ["Un avion est en train de décoller.",
#           "Un homme joue d'une grande flûte.",
#           "Un homme étale du fromage râpé sur une pizza.",
#           "Une personne jette un chat au plafond.",
#           "Une personne est en train de plier un morceau de papier.",
#           ]

# embeddings = model.encode(sentences)

In [42]:
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/dangvantuan/sentence-camembert-large")

from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings
# import os

# os.environ["PINECONE_API_KEY"] = "key lenna"
embedder = HuggingFaceEmbeddings(
    model_name = "EIStakovskii/LaBSE-fr-de"
)

index_name = "pfa"

docsearch = PineconeVectorStore.from_documents(chunks, embedder, index_name=index_name)

In [18]:
# from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

# model = SentenceTransformerEmbeddings(model_name="cointegrated/rubert-tiny2")
# from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
# model =  SentenceTransformerEmbeddings(model_name="dangvantuan/sentence-camembert-large")
from langchain_community.embeddings import HuggingFaceEmbeddings

embedder = HuggingFaceEmbeddings(
    model_name = "cointegrated/rubert-tiny2"
)

persist_directory = 'docs/chroma/'

vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embedder,
    persist_directory=persist_directory
) 

vectordb.persist()
#vectordb.delete_collection()
print(vectordb._collection.count())

42


In [ ]:
index.save_local("faiss_index")
# loaded_index = FAISS.load_local("faiss_index", embeddings)

# Retrieval
## check retriever

In [57]:
# retriever = index.as_retriever(search_kwargs={"k": 3})
# retriever.invoke("quelle est la réligion de la tunisie?")
query="décrivez le drapeau de la tunisie?"
# retriever = docsearch.as_retriever(search_type="mmr")
# matched_docs = retriever.get_relevant_documents(query)
# for i, d in enumerate(matched_docs):
#     print(f"\n## Document {i}\n")
#     print(d.page_content)
found_docs = docsearch.max_marginal_relevance_search(query, k=2, fetch_k=10)
for i, doc in enumerate(found_docs):
    print(f"{i + 1}.", doc.page_content, "\n\n")
# sim_docs = docsearch.similarity_search_with_score(query, k=3)  
# for result in sim_docs:
#     print("\n")
#     print(result[1])
#     print(result[0].page_content)

1. Article 3 
La souveraineté appartient au peuple Tu nisien qui l'exerce conformément à la 
constitution. 
Article 4 
Le drapeau de la République Tunisie nne est rouge, il comporte, dans les 
conditions définies par la loi, en son milie u, un cercle blanc où figure une étoile à 
cinq branches entourée d'un croissant rouge. 
La devise de la République es t : Liberté, Ordre, Justice. 


2. "Je jure par Dieu Tout-Puissant de servir  mon pays loyalement, de respecter la 
Constitution et l'allégeance exclusive envers la Tunisie". 




In [58]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)



new_query= "contexte : "+ format_docs(found_docs)+ "question : "+ query +"répondez en français"



In [59]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2")

llm.invoke(new_query)

" Le drapeau de la Tunisie est rouge et comporte, dans les conditions définies par la loi, un cercle blanc où figure une étoile à cinq branches entourée d'un croissant rouge."

In [ ]:
query = "Tell me a joke"

for chunks in llm.stream(query):
    print(chunks)

S
ure
!
 Here
'
s
 one
:




Why
 don
'
t
 scient
ists
 trust
 atoms
?


B
ecause
 they
 make
 up
 everything
!




I
 hope
 that
 brought
 a
 smile
 to
 your
 face
!
 Do
 you
 have
 any
 specific
 topic
 or
 theme
 in
 mind
 for
 a
 jo
ke
,
 and
 I
 can
 try
 to
 come
 up
 with
 something
 tail
ored
 to
 your
 prefer
ences
?

